# O PROBLEMA

Imagine agora que você foi contratado(a) como Expert em Data Analytics por um grande hospital para entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença.

Apesar de ser contratado(a) agora, a sua área observou que a utilização do estudo do PNAD-COVID 19 do IBGE seria uma ótima base para termos boas respostas ao problema proposto, pois são dados confiáveis.Porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas.

É sempre bom ressaltar que há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

## PNAD-COVID-19 do IBGE

O Head de Dados pediu para que você entrasse na base de dados do PNAD-COVID-19 do IBGE e organizasse esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

- a. Utilização de no máximo 20 questionamentos realizados na pesquisa;
- b. Utilizar 3 meses para construção da solução;
- c. Caracterização dos sintomas clínicos da população;
- d. Comportamento da população na época da COVID-19;
- e. Características econômicas da Sociedade;

Seu objetivo será trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19.

- Dica: Leiam com atenção a base de dados e toda a documentação que o site o PNAD – Covid19 traz, principalmente os dicionários, que ajudam e muito no entendimento da Base de Dados.
- Dica 2: Utilizem o que já foi ensinado e consolidado nas outras fases para apresentar a resolução do projeto.

Lembre-se de que você poderá apresentar o desenvolvimento do seu projeto durante as lives com docentes. Essa é uma boa oportunidade para discutir sobre as dificuldades encontradas e pegar dicas valiosas com especialistas e colegas de turma.

Link para a base: https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=downloads&utm_source=covid19&utm_medium=hotsite&utm_campaign=covid_19

### Importando bibliotecas

In [ ]:
import warnings 
warnings.filterwarnings(action = 'ignore')

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from pyspark.sql import SparkSession
import findspark
import os
import pandas as pd

### Conectando ao MySQL

In [3]:
'''def mysql_connection(host, user, passwd, database=None):
    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')
    return engine.connect()

mysql = mysql_connection('127.0.0.1', 'root', '<senha>', 'pnad_covid')'''

"def mysql_connection(host, user, passwd, database=None):\n    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')\n    return engine.connect()\n\nmysql = mysql_connection('127.0.0.1', 'root', '<senha>', 'pnad_covid')"

In [4]:
'''statement = text('SELECT * FROM b0011;')
response = mysql.execute(statement)
for row in response:
        print(row)'''

"statement = text('SELECT * FROM b0011;')\nresponse = mysql.execute(statement)\nfor row in response:\n        print(row)"

### Criando sessão Spark

In [5]:
os.environ['SPARK_HOME'] = '/opt/spark/'
findspark.init()

In [6]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

23/10/14 22:51:49 WARN Utils: Your hostname, platero-Lenovo-IdeaPad-S145-15IWL resolves to a loopback address: 127.0.1.1; using 192.168.15.109 instead (on interface wlp2s0)
23/10/14 22:51:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/14 22:51:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark

### Importando dados

In [8]:
'''df_05 = spark.read.csv('dados/dados_importados/PNAD_COVID_052020.csv', sep=',', inferSchema=True, header=True)
df_06 = spark.read.csv('dados/dados_importados/PNAD_COVID_062020.csv', sep=',', inferSchema=True, header=True)
df_07 = spark.read.csv('dados/dados_importados/PNAD_COVID_072020.csv', sep=',', inferSchema=True, header=True)
df_08 = spark.read.csv('dados/dados_importados/PNAD_COVID_082020.csv', sep=',', inferSchema=True, header=True)
df_09 = spark.read.csv('dados/dados_importados/PNAD_COVID_092020.csv', sep=',', inferSchema=True, header=True)
df_10 = spark.read.csv('dados/dados_importados/PNAD_COVID_102020.csv', sep=',', inferSchema=True, header=True)
df_11 = spark.read.csv('dados/dados_importados/PNAD_COVID_112020.csv', sep=',', inferSchema=True, header=True)
'''

"df_05 = spark.read.csv('dados/dados_importados/PNAD_COVID_052020.csv', sep=',', inferSchema=True, header=True)\ndf_06 = spark.read.csv('dados/dados_importados/PNAD_COVID_062020.csv', sep=',', inferSchema=True, header=True)\ndf_07 = spark.read.csv('dados/dados_importados/PNAD_COVID_072020.csv', sep=',', inferSchema=True, header=True)\ndf_08 = spark.read.csv('dados/dados_importados/PNAD_COVID_082020.csv', sep=',', inferSchema=True, header=True)\ndf_09 = spark.read.csv('dados/dados_importados/PNAD_COVID_092020.csv', sep=',', inferSchema=True, header=True)\ndf_10 = spark.read.csv('dados/dados_importados/PNAD_COVID_102020.csv', sep=',', inferSchema=True, header=True)\ndf_11 = spark.read.csv('dados/dados_importados/PNAD_COVID_112020.csv', sep=',', inferSchema=True, header=True)\n"

### Tratando dados

In [9]:
'''#padronizando número de colunas
from pyspark.sql.functions import lit

def add_missing_cols(df):
    cols = df_11.columns
    new_cols = []

    for col_name in cols: #selecionando colunas faltantes
        if col_name not in df.columns:
            new_cols.append(col_name)

    for new_col_name in new_cols: #adicionando colunas faltantes
        df = df.withColumn(new_col_name, lit(None))

    return df.select(cols) #organizando colunas

df_05 = add_missing_cols(df_05)
df_06 = add_missing_cols(df_06)
df_07 = add_missing_cols(df_07)
df_08 = add_missing_cols(df_08)
df_09 = add_missing_cols(df_09)
df_10 = add_missing_cols(df_10)'''

'#padronizando número de colunas\nfrom pyspark.sql.functions import lit\n\ndef add_missing_cols(df):\n    cols = df_11.columns\n    new_cols = []\n\n    for col_name in cols: #selecionando colunas faltantes\n        if col_name not in df.columns:\n            new_cols.append(col_name)\n\n    for new_col_name in new_cols: #adicionando colunas faltantes\n        df = df.withColumn(new_col_name, lit(None))\n\n    return df.select(cols) #organizando colunas\n\ndf_05 = add_missing_cols(df_05)\ndf_06 = add_missing_cols(df_06)\ndf_07 = add_missing_cols(df_07)\ndf_08 = add_missing_cols(df_08)\ndf_09 = add_missing_cols(df_09)\ndf_10 = add_missing_cols(df_10)'

In [10]:
'''#concatenando DataFrames
df = df_05.union(df_06)
df = df.union(df_07)
df = df.union(df_08)
df = df.union(df_09)
df = df.union(df_10)
df = df.union(df_11)'''

'#concatenando DataFrames\ndf = df_05.union(df_06)\ndf = df.union(df_07)\ndf = df.union(df_08)\ndf = df.union(df_09)\ndf = df.union(df_10)\ndf = df.union(df_11)'

In [11]:
'''df.show(5)'''

'df.show(5)'

In [12]:
'''df.select('C007C').where(df['V1013'] == '06').show(5)'''

"df.select('C007C').where(df['V1013'] == '06').show(5)"

### Exportando DataFrame para CSV

In [13]:
#exportando arquivo csv único
'''df.coalesce(1).write.csv(
    path='dados/dados_exportados',
    mode='overwrite',
    sep=',',
    header=True
)'''

"df.coalesce(1).write.csv(\n    path='dados/dados_exportados',\n    mode='overwrite',\n    sep=',',\n    header=True\n)"

In [14]:
'''df = spark.read.csv('dados/dados_exportados/PNAD_COVID.csv', sep=',', inferSchema=True, header=True)'''

"df = spark.read.csv('dados/dados_exportados/PNAD_COVID.csv', sep=',', inferSchema=True, header=True)"

In [15]:
'''df.show(5)'''

'df.show(5)'

In [16]:
'''df.count()'''

'df.count()'

In [17]:
#exportando seguimentos do DataFrame
'''n_splits = 5
n_rows = df.count() // n_splits
copy_df = df

for i in range(1, n_splits + 1):
    temp_df = copy_df.limit(n_rows)
    copy_df = copy_df.subtract(temp_df) #subtraindo conteúdo seguimentado
    
    #exportando seguimento
    temp_df.coalesce(1).write.csv(
    path=f'dados/dados_exportados/dados_seguimentados/seguimento_{i}',
    mode='overwrite',
    sep=',',
    header=True
    )'''

"n_splits = 5\nn_rows = df.count() // n_splits\ncopy_df = df\n\nfor i in range(1, n_splits + 1):\n    temp_df = copy_df.limit(n_rows)\n    copy_df = copy_df.subtract(temp_df) #subtraindo conteúdo seguimentado\n    \n    #exportando seguimento\n    temp_df.coalesce(1).write.csv(\n    path=f'dados/dados_exportados/dados_seguimentados/seguimento_{i}',\n    mode='overwrite',\n    sep=',',\n    header=True\n    )"

In [19]:
df_1 = spark.read.csv('dados/dados_exportados/dados_seguimentados/seguimento_1/*.csv', sep=',', inferSchema=True, header=True)
df_2 = spark.read.csv('dados/dados_exportados/dados_seguimentados/seguimento_2/*.csv', sep=',', inferSchema=True, header=True)

In [20]:
df_1.show(5)

23/10/14 22:53:27 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+---+-------+-------+-----+-----+-----+-----+-------+---------+-----+-----+------+-----------+------------+------+----+-----+------+------+------+----+----+----+----+----+-----+-----+----+-----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+----+----+-----+-----+-----+----+----+-----+-----+-----+-----+-----+------+------+-----+----+----+-----+----+-----+------+------+-----+------+------+-----+-----+-----+------+-------+-------+------+-------+-------+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+----+-----+-----+------+------+------+------+------+-----+----+
| Ano| UF|CAPITAL|RM_RIDE|V1008|V1012|V1013|V1016|Estrato|      UPA|V1022|V1023| V1030|      

In [21]:
df_2.show(5)

+----+---+-------+-------+-----+-----+-----+-----+-------+---------+-----+-----+------+------------+------------+------+----+-----+------+------+------+----+----+----+----+----+-----+-----+----+-----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+----+----+-----+-----+-----+----+----+-----+-----+-----+-----+-----+------+------+-----+----+----+-----+----+-----+------+------+-----+------+------+-----+-----+-----+------+-------+-------+------+-------+-------+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+----+-----+-----+------+------+------+------+------+-----+----+
| Ano| UF|CAPITAL|RM_RIDE|V1008|V1012|V1013|V1016|Estrato|      UPA|V1022|V1023| V1030|     

In [ ]:
'''df.createOrReplaceTempView('view')'''

In [22]:
'''spark.sql('SELECT ANO, V1023, CAPITAL FROM view WHERE V1013 = 05 LIMIT 5').toPandas()'''

"spark.sql('SELECT ANO, V1023, CAPITAL FROM view WHERE V1013 = 05 LIMIT 5').toPandas()"

### Exportando dados para o MySQL